## 앙상블 학습
여러 개의 분류기(Classifier)를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법을 말합니다.

- 비정형 데이터의 분류는 딥러닝이 뛰어난 성능을 보이고 있지만, 대부분의 정형 데이터 분류는 앙상블이 뛰어나 성능을 나타낸다.

### 보팅 
한 데이터를 기준으로 서로 다른 알고리즘의 분류기들을 결합하여 최종 예측(Voting)하는 것.

### 배깅
각각의 데이터들을 기준으로 같은 유형의 알고리즘을 기반으로 학습을 수행한 뒤에 예측 값들을 결합하여 최종 예측(Voting)하는 것.

### 부스팅
여러개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린데이터에 대해서는 올바르게 예측할 수 있도록 다음 분류기에게는 가중치를 부여하면서 학습과 예측을 진행하는 것입니다.

### 하드보팅
다수결로 높은 예측값(이진분류)으로 최종 class를 결정한다.

### 소프트보팅
각각의 확률 예측 값들의 평균을 확인하여 class 값을 예측하여 결정한다. 


In [1]:
# 보팅 분류기 앙상블 구현 실습
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cancer = load_breast_cancer(as_frame=True)
# 기본 전처리 돼있고 숫자로 처리 돼있음.

In [3]:
cancer.data.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [4]:
cancer.target.head(2)

# 악성이 : 0
# 양성이 : 1

0    0
1    0
Name: target, dtype: int32

In [5]:
cancer.target_names

array(['malignant', 'benign'], dtype='<U9')

In [6]:
# 보팅 분류기 사용했을 경우

lr_clf = LogisticRegression(solver='liblinear')
# n_neighbors = 5 : 주변 가까운 데이터 8개를 기준으로 ? 작업 수행
knn_clf = KNeighborsClassifier(n_neighbors=8)
# 리스트 안에 튜플 형태가 된다.
vo_clf = VotingClassifier([('lr',lr_clf),('knn',knn_clf)],voting='soft')
X_train,X_test,y_train,y_test = train_test_split(cancer.data,
                                                 cancer.target,
                                                 test_size=0.2,
                                                 random_state=156)
vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
accuracy_score(y_test,pred)

0.956140350877193

In [7]:
# 개별 모델만 사용했을 경우
models = [lr_clf,knn_clf]
for model in models:
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    model_name = model.__class__.__name__
    print(f'{model_name} 정확도 : {accuracy_score(y_test,pred)}')

LogisticRegression 정확도 : 0.9473684210526315
KNeighborsClassifier 정확도 : 0.9385964912280702


C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## 랜덤 포레스트
배깅의 대표적인 알고리즘

### 랜덤 포레스트 하이퍼 파라미터 및 튜닝
- 단점 
1. 하이퍼 파라미터가 너무 많다.
2. 튜닝 시간이 너무 많이 소모된다.
3. 튜닝 후 예측 성능이 크게 향상되는 경우가 많지 않다.

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
# 중복 되는 것들 일렬 번호 매겨주는 함수 생성
def get_new_df(old_df):
    dup_df = pd.DataFrame(data=old_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    dup_df = dup_df.reset_index()
    # 같은 인덱스끼리 연결해서 뒤에 dup_cnt 추가해줌.
    new_df = pd.merge(old_df.reset_index(),dup_df,how='outer')
    # 가져올 컬럼 두개 선택함. 
    # 행을 기준으로 묶어서 들어오는 경우는 axis 기본값(0) 일 경우.
    # axis 값 1로 주면 행을 기준으로 들어와서 하나씩 들어와짐.
    # 이렇게 해야 이 컬럼 값의 뒤에 `_`를 연결시켜서 값을 넣어줌.
    # 파이썬에선 데이터 타입이 같아야 연산이 가능함.
    new_df['column_name'] = new_df[['column_name','dup_cnt']].apply(lambda X:X[0]+'_'+str(X[1]) if X[1]>0 else X[0], axis=1)
    new_df.drop(columns=['index'],inplace=True)
    
    return new_df

In [10]:
def get_human_dataset():
    feature_name_df = pd.read_csv('human_activity/features.txt',
                            sep='\s+',
                            header=None,
                            names=['column_index','column_name'])
    name_df = get_new_df(feature_name_df)
    feature_name = name_df.iloc[:,1].values.tolist()
    X_train = pd.read_csv('human_activity/train/X_train.txt',sep='\s+',names=feature_name)
    X_test = pd.read_csv('human_activity/test/X_test.txt',sep='\s+',names=feature_name)
    y_train = pd.read_csv('human_activity/train/y_train.txt',sep='\s+',names=['action'])
    y_test = pd.read_csv('human_activity/test/y_test.txt',sep='\s+',names=['action'])
    return X_train,X_test,y_train,y_test

In [11]:
X_train,X_test,y_train,y_test = get_human_dataset()

In [12]:
# `n_estimators=100,` : 기본값 100, 결정트리 100개로 학습?
# depth 수정하면 정확도 달라짐
dt_clf = RandomForestClassifier(random_state=0,max_depth=16)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
accuracy_score(y_test,pred)

C:\Users\user\AppData\Local\Temp\ipykernel_7892\1640099684.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  dt_clf.fit(X_train,y_train)


0.9253478113335596

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
params ={
    'max_depth' : [8,16,24],
    'min_samples_split' : [2,8,16],
    'min_samples_leaf' : [1,6,12],
}

In [15]:
%%time
# `n_jobs=-1` 모든 프로세스를 사용하는 것으로 지정하게 된다.
rf_clf = RandomForestClassifier(random_state=0,n_jobs=-1)
grid_cv = GridSearchCV(rf_clf,params,cv=2,n_jobs=-1)
grid_cv.fit(X_train,y_train)

KeyboardInterrupt: 

In [16]:
grid_cv.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_cv.best_score_

In [ ]:
# 결정 트리 조건 파라미터 추가 했더니 정확도 향상 됨
dt_clf = RandomForestClassifier(random_state=0,max_depth=16,min_samples_leaf=6,min_samples_split=2)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
accuracy_score(y_test,pred)